# Лаб 2. Матричные разложения

## Теория

Суть методов, основанных на матричном разложении состоит в том, что каждому пользователю $u$ ставится в соответствие вектор интересов $p_u$ длины $k$, а каждому айтему $i$ ставится в соответствие вектор интересов $q_i$, которым он удовлетворяет. Таким образом, что скалярное произведение этих векторов даёт предсказание $\hat r_{ui}$:

$$
\hat r_{ui} = p_u \dot q_i
$$

Такие вектора $p_u$ и $q_i$ составляют матрицы $P$ и $Q$ соответственно, которые в произведении даёт матрицу предсказаний $\hat R$.

![Разложение матриц](./images/PQ.drawio.png)

## Код

Ставим библиотеки

In [1]:
#%pip install --quiet -U scikit-surprise

Импорты

In [2]:
from tqdm.notebook import tqdm
import pandas as pd

# Реализации методов матричного разложения будем использовать из библиотеки surprise
from surprise import Dataset, SVD, NMF, Reader
# Кроссвалидацию проходили на машинном обучении
from surprise.model_selection import cross_validate

Загрузка датасета (из первой лабораторной)

In [3]:
data = Dataset.load_from_file('./ml-latest-small/ratings.csv', Reader(sep=',', line_format="user item rating timestamp", skip_lines=1))
trainset = data.build_full_trainset()

user_id = '42'
item_id = '50'

### SVD

In [5]:
# Обучаем модель
algo_svd = SVD(biased=False, n_factors=100)
algo_svd = algo_svd.fit(trainset)

In [6]:
# Даём предсказание для юзера и произвольного айтема
algo_svd.predict(user_id, item_id).est

4.278161972429333

Матричное представление пользователей

In [7]:
algo_svd.pu.shape

(610, 100)

In [8]:
#algo_svd.pu

Матричное представление товаров

In [9]:
algo_svd.qi.shape

(9724, 100)

In [10]:
#algo_svd.qi

Пример связи матричных представлений и итогово предсказания

In [11]:
svd_item_id = algo_svd.trainset.to_inner_iid(item_id)
svd_user_id = algo_svd.trainset.to_inner_uid(user_id)

# Умножаем вектор интересов пользователя на вектор интересов айтема
(algo_svd.pu[svd_user_id]*algo_svd.qi[svd_item_id]).sum()

4.278161972429334

In [12]:
algo_svd.pu[svd_user_id].shape,algo_svd.qi[svd_item_id].shape,svd_item_id,svd_user_id

((100,), (100,), 4, 41)

### Non-negative Matrix Factorization

По интерфейсу и внутреннему устройству этот метод похож на SVD. Повторите предыдущие шаги для него.

In [13]:
algo_nmf = NMF(n_factors=15)
algo_nmf = algo_nmf.fit(trainset)

In [14]:
algo_nmf.predict(user_id, item_id).est

4.327444417140317

In [15]:
nmf_item_id = algo_nmf.trainset.to_inner_iid(item_id)
nmf_user_id = algo_nmf.trainset.to_inner_uid(user_id)

# Умножаем вектор интересов пользователя на вектор интересов айтема
(algo_nmf.pu[nmf_user_id]*algo_nmf.qi[nmf_item_id]).sum()

4.327444417140317

### Валидация и обучение
Используя кроссвалидацию, сравните представленные алгоритмы по качеству.

Подберите гиперпараметры моделей для улучшения результатов.

Расшифровка метрик ошибок и альтернативы можно найти в документации (как и параметры моделей): https://surprise.readthedocs.io/en/stable/accuracy.html#module-surprise.accuracy

In [16]:
cross_validate(algo_svd, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9763  0.9838  0.9780  0.9790  0.9846  0.9804  0.0033  
MAE (testset)     0.7506  0.7569  0.7562  0.7530  0.7566  0.7547  0.0025  
Fit time          1.20    1.14    1.13    1.19    1.13    1.16    0.03    
Test time         0.24    0.11    0.13    0.20    0.12    0.16    0.05    


{'test_rmse': array([0.97629827, 0.98383663, 0.97799109, 0.97902917, 0.98461186]),
 'test_mae': array([0.75057904, 0.7569088 , 0.75623263, 0.75303244, 0.75657811]),
 'fit_time': (1.2007877826690674,
  1.1449074745178223,
  1.1339657306671143,
  1.1868233680725098,
  1.1299760341644287),
 'test_time': (0.23636722564697266,
  0.11369562149047852,
  0.1296532154083252,
  0.19752168655395508,
  0.1216742992401123)}

In [17]:
cross_validate(algo_nmf, data, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9292  0.9199  0.9189  0.9172  0.9192  0.9209  0.0042  
MAE (testset)     0.7115  0.7031  0.7049  0.7015  0.7035  0.7049  0.0034  
Fit time          2.38    2.29    2.33    2.24    2.19    2.28    0.07    
Test time         0.12    0.21    0.12    0.11    0.19    0.15    0.04    


{'test_rmse': array([0.92920069, 0.91993192, 0.91891917, 0.91721236, 0.91923552]),
 'test_mae': array([0.71145618, 0.70307095, 0.70491371, 0.70154439, 0.70351632]),
 'fit_time': (2.3786356449127197,
  2.285883665084839,
  2.328768730163574,
  2.2350199222564697,
  2.1901400089263916),
 'test_time': (0.11868143081665039,
  0.20844316482543945,
  0.11668777465820312,
  0.11170077323913574,
  0.18650174140930176)}

In [18]:
from tqdm.notebook import tqdm
svd_rmse={}
start=0
stop=200
step=1
'''
with tqdm(total=(stop-start)/step) as pbar:
    for i in range(start,stop,step):
        algo_svd = SVD(biased=False, n_factors=i)
        algo_svd = algo_svd.fit(trainset)
        svd_rmse[i]=cross_validate(algo_svd, data, verbose=False)['test_rmse'].mean()
        pbar.update(1)
dict(sorted(svd_rmse.items(), key=lambda item: item[1], reverse=False))
'''

"\nwith tqdm(total=(stop-start)/step) as pbar:\n    for i in range(start,stop,step):\n        algo_svd = SVD(biased=False, n_factors=i)\n        algo_svd = algo_svd.fit(trainset)\n        svd_rmse[i]=cross_validate(algo_svd, data, verbose=False)['test_rmse'].mean()\n        pbar.update(1)\ndict(sorted(svd_rmse.items(), key=lambda item: item[1], reverse=False))\n"

In [19]:
nmf_rmse={}
start=0
stop=200
step=1
'''
with tqdm(total=(stop-start)/step) as pbar:
    for i in range(start,stop,step):
        algo_nmf = SVD(biased=False, n_factors=i)
        algo_nmf = algo_svd.fit(trainset)
        nmf_rmse[i]=cross_validate(algo_nmf, data, verbose=False)['test_rmse'].mean()
        pbar.update(1)
dict(sorted(nmf_rmse.items(), key=lambda item: item[1], reverse=False))
'''

"\nwith tqdm(total=(stop-start)/step) as pbar:\n    for i in range(start,stop,step):\n        algo_nmf = SVD(biased=False, n_factors=i)\n        algo_nmf = algo_svd.fit(trainset)\n        nmf_rmse[i]=cross_validate(algo_nmf, data, verbose=False)['test_rmse'].mean()\n        pbar.update(1)\ndict(sorted(nmf_rmse.items(), key=lambda item: item[1], reverse=False))\n"

In [20]:
algo_svd = SVD(biased=False, n_factors=8)
algo_svd = algo_svd.fit(trainset)
cross_validate(algo_svd, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9354  0.9371  0.9443  0.9349  0.9256  0.9355  0.0060  
MAE (testset)     0.7142  0.7160  0.7216  0.7121  0.7114  0.7151  0.0037  
Fit time          0.51    0.52    0.55    0.56    0.57    0.54    0.02    
Test time         0.19    0.09    0.22    0.11    0.10    0.14    0.05    


{'test_rmse': array([0.93542255, 0.93705772, 0.9442981 , 0.93486452, 0.92564707]),
 'test_mae': array([0.71423073, 0.71598943, 0.7216143 , 0.71205368, 0.71136274]),
 'fit_time': (0.5146231651306152,
  0.5186386108398438,
  0.5535533428192139,
  0.5585057735443115,
  0.5704412460327148),
 'test_time': (0.18752741813659668,
  0.09471797943115234,
  0.21542978286743164,
  0.1137399673461914,
  0.10475897789001465)}

In [21]:
algo_nmf = NMF(biased=False,n_factors=23)
algo_nmf = algo_nmf.fit(trainset)
cross_validate(algo_nmf, data, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9313  0.9319  0.9225  0.9310  0.9118  0.9257  0.0078  
MAE (testset)     0.7025  0.7071  0.6967  0.7057  0.6920  0.7008  0.0057  
Fit time          2.68    3.02    3.15    2.82    2.69    2.87    0.18    
Test time         0.20    0.11    0.11    0.20    0.11    0.15    0.04    


{'test_rmse': array([0.93135   , 0.93190958, 0.92251375, 0.93095241, 0.91177407]),
 'test_mae': array([0.70249315, 0.70712417, 0.69670001, 0.70567731, 0.69199083]),
 'fit_time': (2.683819055557251,
  3.0209155082702637,
  3.1495721340179443,
  2.8234150409698486,
  2.6917970180511475),
 'test_time': (0.20046329498291016,
  0.1107032299041748,
  0.11369609832763672,
  0.20046234130859375,
  0.11170220375061035)}

## Призрак в машине

Проинтерпретируйте получившиеся "интересы" в модели NnMF.

Например, можно начать с просмотра отсортированных по ним фильмов

In [23]:
movies = pd.read_csv('./ml-latest-small/movies.csv', delimiter=',')
# Делаем таблицу для преобразования id в имя
id_to_title = movies.loc[:, ["movieId", "title"]]
id_to_title.set_index("movieId", inplace=True)

In [24]:
factor_idx = 0
items = list(algo_nmf.trainset.all_items())

items.sort(key=lambda inner_id: -algo_nmf.qi[inner_id][factor_idx])
named_items = [[id_to_title.loc[int(trainset.to_raw_iid(k))]["title"], [int(f*100) for f in algo_nmf.qi[k]]] for k in items]
#named_items

In [25]:
for i in range (5):
    print(named_items[i][0],'\t\t',movies.loc[(movies['title'] == named_items[i][0])]['genres'].reset_index(drop=True)[0])
print("-----------")
for i in range (5,0,-1):
    print(named_items[-i][0],'\t\t',movies.loc[(movies['title'] == named_items[-i][0])]['genres'].reset_index(drop=True)[0])  

The Adventures of Sherlock Holmes and Dr. Watson: The Hound of the Baskervilles (1981) 		 Crime|Mystery
District 13: Ultimatum (Banlieue 13 - Ultimatum) (2009) 		 Action|Sci-Fi
Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972) 		 Adventure|Drama
Dr. Horrible's Sing-Along Blog (2008) 		 Comedy|Drama|Musical|Sci-Fi
Death Wish 2 (1982) 		 Action|Drama
-----------
Billabong Odyssey (2003) 		 Documentary
Cinderella Story, A (2004) 		 Comedy|Romance
In Dreams (1999) 		 Horror|Thriller
Sound of Thunder, A (2005) 		 Action|Adventure|Drama|Sci-Fi|Thriller
Little Boxes (2017) 		 Comedy|Drama


## вывод
разделение по первому признаку разделяет выборку явно, с положительным знаком преимущественно комедии, детские и семейные фильмы, а с отрицательным знаком преимущественно драмы.

## Адаптация собственного алгоритма

Адаптируйте ваш алгоритм из первой лабораторной для совместимости с библиотекой согласно документации: https://surprise.readthedocs.io/en/stable/building_custom_algo.html

Сравните настроенные ранее модели со своим решением с помощью кроссвалидации.

In [235]:
from surprise import AlgoBase, Dataset
from surprise.model_selection import cross_validate
from surprise import PredictionImpossible
f=int(0)
class MyOwnAlgorithm(AlgoBase):
    
    def __init__(self):
        AlgoBase.__init__(self)
        self.f=0
        self.count=0
        self.s={}
        self.u_mid ={}
        self.ui_filmid={}
        self.ui_value={}   
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        if self.f==0:
            for key, value in trainset.ur.items() :
                self.ui_filmid[key]=[]
                self.ui_value[key]=[]
                for i in range (len(trainset.ur[key])):
                    self.ui_filmid[key].append( value[i][0])
                    self.ui_value[key].append(value[i][1])
            for key, value in trainset.ur.items() :
                s_small={}
                sum_u=0
                s_downk=0
                s_downl=0
                for k in range (len(trainset.ur[key])):
                    sum_u+=trainset.ur[key][k][1]
                s_downk= sum(i*i for i in self.ui_value[key])
                for l, e in trainset.ur.items():
                    if (l!=key):
                        s_up=0
                        #print(key, l)
                        s_downl=sum(i*i for i in self.ui_value[l])
                        a= list(set( self.ui_filmid[key])& set(self.ui_filmid[l]))
                        for j in a:
                            s_up+=self.ui_value[key][self.ui_filmid[key].index(j)]*self.ui_value[l][self.ui_filmid[l].index(j)]
                        s_small[l]=s_up/((s_downk*s_downl)**0.5) 
                self.s[key]=s_small
                self.u_mid[key]=sum_u/len(trainset.ur[key])
            self.f=+1
            
        return self
    def estimate(self, u, i):
        self.count+=1
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible("User and/or item is unknown.")
        r_up=0
        r_down=0
        for l, e in trainset.ur.items():
            if (l!=u):
                a= list(set( self.ui_filmid[u])& set(self.ui_filmid[l]))
                if a:
                    for j in a:
                        r_up+=self.s[u][l]*(self.ui_value[l][self.ui_filmid[l].index(j)]-self.u_mid[l])
                        r_down+=abs(self.s[u][l])
        if r_down==0:
            bsl=0.0
        else:
            bsl = self.u_mid[l]+r_up/r_down
        if self.count%1000==0:
            print (self.f,self.count)
        return bsl

In [236]:
#data = Dataset.load_builtin("ml-100k")
algo = MyOwnAlgorithm()


cross_validate(algo, data, verbose=True)

1 1000
1 2000
1 3000
1 4000
1 5000
1 6000
1 7000
1 8000
1 9000
1 10000
1 11000
1 12000
1 13000
1 14000
1 15000
1 16000
1 17000
1 18000
1 19000
1 20000
1 21000
1 22000
1 23000
1 24000
1 25000
1 26000
1 27000
1 28000
1 29000
1 30000
1 31000
1 32000
1 33000
1 34000
1 35000
1 36000
1 38000
1 39000
1 40000
1 41000
1 42000
1 43000
1 44000
1 45000
1 46000
1 47000
1 48000
1 49000
1 50000
1 51000
1 52000
1 53000
1 54000
1 55000
1 56000
1 57000
1 58000
1 59000
1 60000
1 61000
1 63000
1 64000
1 65000
1 66000
1 67000
1 68000
1 69000
1 70000
1 71000
1 72000
1 73000
1 74000
1 75000
1 76000
1 77000
1 78000
1 80000
1 81000
1 82000
1 83000
1 84000
1 85000
1 86000
1 87000
1 88000
1 89000
1 90000
1 91000
1 92000
1 93000
1 94000
1 95000
1 96000
1 97000
1 98000
1 100000
Evaluating RMSE, MAE of algorithm MyOwnAlgorithm on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0399  1.0551  1.0451  1.0515  1.0479  1.0479  0.0052  
MAE (testset)     0.8150 

{'test_rmse': array([1.03985567, 1.05505565, 1.04513776, 1.05153595, 1.04791676]),
 'test_mae': array([0.81503005, 0.82968399, 0.81909215, 0.82611232, 0.82355393]),
 'fit_time': (30.839818954467773,
  0.01994466781616211,
  0.02094554901123047,
  0.0219419002532959,
  0.02094435691833496),
 'test_time': (1329.490199804306,
  798.0431282520294,
  790.1037137508392,
  804.8991403579712,
  804.0147337913513)}

Evaluating RMSE, MAE of algorithm MyOwnAlgorithm on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0399  1.0551  1.0451  1.0515  1.0479  1.0479  0.0052  
MAE (testset)     0.8150  0.8297  0.8191  0.8261  0.8236  0.8227  0.0052  
Fit time          30.84   0.02    0.02    0.02    0.02    6.18    12.33   
Test time         1329.49 798.04  790.10  804.90  804.01  905.31  212.16  
{'test_rmse': array([1.03985567, 1.05505565, 1.04513776, 1.05153595, 1.04791676]),
 'test_mae': array([0.81503005, 0.82968399, 0.81909215, 0.82611232, 0.82355393]),
 'fit_time': (30.839818954467773,
  0.01994466781616211,
  0.02094554901123047,
  0.0219419002532959,
  0.02094435691833496),
 'test_time': (1329.490199804306,
  798.0431282520294,
  790.1037137508392,
  804.8991403579712,
  804.0147337913513)}

In [240]:
cross_validate(algo_svd, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9456  0.9280  0.9352  0.9341  0.9308  0.9347  0.0060  
MAE (testset)     0.7222  0.7089  0.7125  0.7131  0.7141  0.7142  0.0044  
Fit time          0.52    0.55    0.50    0.61    0.52    0.54    0.04    
Test time         0.92    0.11    0.10    0.12    0.10    0.27    0.32    


{'test_rmse': array([0.94561452, 0.92802016, 0.9352116 , 0.93410154, 0.9307723 ]),
 'test_mae': array([0.72220608, 0.70890817, 0.712548  , 0.71313194, 0.71408665]),
 'fit_time': (0.5206050872802734,
  0.5515251159667969,
  0.49767422676086426,
  0.6124670505523682,
  0.5235979557037354),
 'test_time': (0.9165496826171875,
  0.1077108383178711,
  0.09573984146118164,
  0.12164545059204102,
  0.10172772407531738)}

In [241]:
cross_validate(algo_nmf, data, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9281  0.9305  0.9279  0.9130  0.9238  0.9247  0.0062  
MAE (testset)     0.6974  0.7040  0.7036  0.6909  0.7008  0.6993  0.0048  
Fit time          2.72    2.52    2.43    2.41    2.45    2.51    0.11    
Test time         0.10    0.09    0.39    0.09    0.09    0.15    0.12    


{'test_rmse': array([0.92805298, 0.93047094, 0.92794418, 0.91301933, 0.92377845]),
 'test_mae': array([0.69736842, 0.70402294, 0.70355104, 0.69094004, 0.70081859]),
 'fit_time': (2.716731071472168,
  2.5152223110198975,
  2.4285459518432617,
  2.4135797023773193,
  2.4527881145477295),
 'test_time': (0.1047220230102539,
  0.09375238418579102,
  0.3859682083129883,
  0.09474921226501465,
  0.09371805191040039)}

In [237]:
algo.predict(user_id, item_id).est

3.804151635478294

In [238]:
algo_nmf.predict(user_id, item_id).est

4.321548258631787

In [239]:
algo_svd.predict(user_id, item_id).est

4.693973494063265

# Вывод
ошибка полученной модели примерно такая же, как при стандартных значениях начальной модели (n_factors default), но итоговый результат не так хорош, как хотелось бы